In [ ]:
import glob
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import loadmat
from scipy.fftpack import fft
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

# Transformation functions

In [ ]:
def apply_fft(data):
    return np.fft.rfft(data, axis=1)

def mat_to_pd(path):
    mat = loadmat(path)
    names = mat['dataStruct'].dtype.names
    ndata = {n: mat['dataStruct'][n][0, 0] for n in names}
    return pd.DataFrame(ndata['data'], columns=ndata['channelIndices'][0])

def get_fft_bins_edges(pd):
    bins  = []
    edges = []
    for i in range(1, 17):
        freq = np.abs(np.fft.fft(pd[float(str(i) + ".0")]))
        h = np.histogram(np.log(freq).ravel(), bins=100)
        bins.append(h[0])
        edges.append(h[1])
    return np.array(bins), np.array(edges)

def get_fft_bins(pd):
    return get_fft_bins_edges(pd)[0]
    
def get_fft_edges(pd):
    return get_fft_bins_edges(pd)[1]

def get_high(data, return_size=50):
    FFT = abs(scipy.fft(data))
    return FFT[1:return_size + 1]

def get_fft_peaks(data, return_size=50):
    record_trans = np.zeros((data.shape[0], return_size*16))
    for i, record in enumerate(data):
        if not i % 100:
            print(i)
        for channel_index in range(0, 16):
            fft = get_high(record[:, channel_index], return_size=return_size)
            begin_idx = channel_index * return_size
            end_idx  = begin_idx + return_size
            record_trans[i, begin_idx : end_idx] = fft
    return record_trans

# Utility functions

In [ ]:
def save_npz(column_number, name="", phase="train", transform_method=lambda x: x):
    file_list = sorted(glob.glob("./%s_%d/*.mat" % (phase, column_number)))
    print(file_list)
    X = []
    if phase == "train":
        y = []
    for file in file_list:
        filename = os.path.basename(file)[:-4].split("_")
        try:
            data = mat_to_pd(file)
            X.append(transform_method(data))
            if phase == "train":
                y.append(filename[2])
        except Exception as e:
            print("error " + str(e))
            continue

    X_np = np.zeros((len(X), X[0].shape[0], X[0].shape[1]))
    if phase == "train":
        y_np = np.zeros((len(X), ))
    for i in range(len(X)):
        X_np[i] = X[i]
        if phase == "train":
            y_np[i] = y[i]
    if phase == "train":
        np.savez_compressed(file="./data/%s_%s_%d.npz" \
                            % (phase, name, column_number), X=X_np, y=y_np)
    else:
        np.savez_compressed(file="./data/%s_%s_%d.npz" \
                            % (phase, name, column_number), X=X_np)
        
        
def load_data(filename):
    Xy_dict = np.load(filename)
    return tuple(Xy_dict.items())
    
    
# transform_method is applied to data from .mat
# elem - element from tuple, returned after transform_method
# deprecated
def create_csv(data, transform_method=None, elem=None, elem_length=None):
    file_list = sorted(glob.glob("./train_*/*.mat"))
    out = open("./train.csv", "w")
    out.write("id,pid")
    if data is not None:
        data_range = len(data)
    if elem is not None:
        data_range = elem_length*16
    for i in range(data_range):
        out.write(",d" + str(i))
    out.write(",result\n")
    for file in file_list:
        filename = os.path.basename(file)[:-4].split("_")
        patient_id = filename[0]
        id = 100000*patient_id + filename[1]
        result = filename[2]
        try:
            data = mat_to_pd(file)
            if transform_method is not None:
                data = transform_method(data)[elem]
        except Exception as e:
            continue
        out.write(str(id))
        out.write("," + str(patient_id))
        for datum in data:
            for element in datum:
                out.write("," + str(element))
        out.write("," + str(result))
        out.write("\n")
    out.close

# Learning

## Trying to make FFT histogram bins work

In [ ]:
create_csv(None, get_fft_bins_edges, 0, 100)
# save_npz(100, get_fft_bins)
data = pd.read_csv("./train.csv")
del(data['id'])
X = data.drop(['result', 'pid'], axis=1)
y = data['result']

In [ ]:
pca = PCA()
pca.fit(X)
X_transformed = pca.transform(X)
plt.plot(X_transformed[np.where(y == 0)[0], 4], X_transformed[np.where(y == 0)[0], 5], 'ro')
plt.plot(X_transformed[np.where(y == 1)[0], 4], X_transformed[np.where(y == 1)[0], 5], 'bo')

In [ ]:
# Checking performance 
clas = RandomForestClassifier(max_depth=4, n_estimators=1000)    
cross_val_score(clas, X, y, scoring="roc_auc", cv=5)

## Getting raw FFT results

In [ ]:
pca = {}
FFTX = {}
FFTXPCA = {}
classifiers = {}

for i in range(1, 4):
   X  = load_data(os.path.join(sys.argv[1], "data/test_identity_%d.npz" % i))
   FFTX[i] = get_fft_peaks(X[0], return_size=200)
   pca[i] = PCA()
   FFTXPCA[i] = pca[i].fit_transform(FFTX[i])

   indices = np.arange(FFTXPCA[i].shape[0])
   np.random.shuffle(indices)
   train_fft, test_fft = indices[:indices.shape[0]*0.8], indices[indices.shape[0]*0.8:]

   classifiers[i] = XGBClassifier()
   classifiers[i].fit(FFTXPCA[i][train_fft], y[train_fft])
    np.savez_compressed(file=os.path.join(sys.argv[1], "data/test_fft_simple_%d_%d.npz" % (i, 200)), X=FFTX[i] )
   print(classifiers[i].score(FFTXPCA[i][test_fft], y[test_fft]))

pickle.dump(classifiers, open(os.path.join(sys.argv[1], "models/2016-09-14"), "wb"))